https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/intro.ipynb#scrollTo=4SEChu_DmoGt

### Loading API Keys
Set the environment `OPENAI_API_KEY in a .env file which we will be loading using dotenv.

In [ ]:
import dotenv
dotenv.load_dotenv("../.env", override=True)

### Setting up the model
We will start by initializing the language model. The model can be easily swapped here for API or local models

In [ ]:
import dspy

lm = dspy.OpenAI(model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

### DSPy fundamentals

In [ ]:
question = "Who was the first person to walk on the moon?"

qa = dspy.Predict('question -> piglatin')
response = qa(question=question)

print(response.piglatin)

### Signature

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

predictor = dspy.ChainOfThought(BasicQA)
result = predictor(
    question=("What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?")
)
print(result)

In [ ]:
# lm.inspect_history()

### Module

In [ ]:
class QA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(BasicQA)
    
    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return prediction
    
qa = QA()
result = qa(
    question=("What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?")
)
print(result)

#### Datasets/Examples

In [ ]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

In [ ]:
train_example = trainset[0]
train_example

### RAG

In [ ]:
retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(train_example.question).passages

print(f"Top {retrieve.k} passages for question: {train_example.question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

rag = RAG()
rag(train_example.question)

### Bootstrap

In [ ]:
from dspy.teleprompt import BootstrapFewShot

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

In [ ]:
pred = compiled_rag("What castle did David Gregory inherit?")

print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {pred.context}")

In [ ]:
# lm.inspect_history(n=1)

### Evaluation

In [ ]:
from dspy.evaluate.evaluate import Evaluate

evaluate_on_hotpotqa = Evaluate(devset=devset[:2], num_threads=1, display_progress=True, display_table=5)

metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)